# GET Airports for cities in europe

In [13]:
import pandas as pd
import requests
import json
import load_dotenv
import os
import time

In [14]:
# Environment-variables in .env-Datei
load_dotenv()

# API-Key
api_key = os.getenv("AVIATION_KEY")

# API URL
base_url = "https://aviation-edge.com/v2/public/autocomplete"

In [15]:
# airports in europe csv
airports_df = pd.read_csv("wikipedia_airports_eu.csv")

In [16]:
city_country_pairs = airports_df[['City served', 'Country']].head(20).drop_duplicates()

In [17]:
city_data = {}

# Loop through each unique city-country pair
for _, row in city_country_pairs.iterrows():
    city = row['City served']
    country = row['Country']
    
    # Define the API request URL
    url = f"{base_url}?key={api_key}&city={city}"
    
    # Make the request to the API
    response = requests.get(url)
    
    if response.status_code == 200:
        response_data = response.json().get('airportsByCities', [])
        
        # Filter results where nameCountry matches the 'Country' from the CSV
        filtered_data = [item for item in response_data if item.get('nameCountry') == country]
        
        # Store only if there are matching results
        if filtered_data:
            city_data[city] = filtered_data
            print(f"Data for {city} in {country} retrieved successfully.")
        else:
            print(f"No matching data for {city} in {country}.")
    
    else:
        print(f"Failed to retrieve data for {city}. Status code: {response.status_code}")
    
    time.sleep(1)

Data for London in United Kingdom retrieved successfully.
Data for Istanbul in Turkey retrieved successfully.
Data for Paris in France retrieved successfully.
Data for Amsterdam in Netherlands retrieved successfully.
Data for Madrid in Spain retrieved successfully.
Data for Frankfurt in Germany retrieved successfully.
Data for Barcelona in Spain retrieved successfully.
Data for Rome in Italy retrieved successfully.
Data for Munich in Germany retrieved successfully.
Data for Moscow in Russia retrieved successfully.
Data for Lisbon in Portugal retrieved successfully.
Data for Dublin in Ireland retrieved successfully.
Data for Palma Mallorca in Spain retrieved successfully.
Data for Vienna in Austria retrieved successfully.
Data for Zurich in Switzerland retrieved successfully.
Data for Athens in Greece retrieved successfully.
Data for Manchester in United Kingdom retrieved successfully.


In [18]:
for city, airports in city_data.items():
    # Filter airports list within each city entry
    city_data[city] = [airport for airport in airports if airport.get("codeIcaoAirport")]

In [19]:
with open("airport_per_city.json", "w") as outfile:
    json.dump(city_data, outfile)